# Tutorial. The Chebyshev interpolation

Spectral methods are a class of spatial discretisation methods for differential equations in which the approximation of the solution $u$ of the problem is based an expansion in terms of so-called *trial functions* $\{\phi_k\}_{k=0,\dots,N-1}$,
$$
u(x)\approx\sum_{k=0}^{N-1}\tilde{u}_k\phi_k(x),
$$
the coefficients of the expansion being noted $\tilde{u}_k$, $k=0,\dots,N-1$.

The choice of the trial function is dictated by the practical and computational efficiency of of the numerical method, and it has to meet the following requirements:
* *Convergence:* the approximation should converge rapidly to the solution $u$ as $N$ tends to $+\infty$,
* *Transformation:* the computation of the coefficients $\tilde{u}_k$ from the values of $u$ and the reconstruction of the function values at given nodes from the set of coefficients should be computationally fast,
* *Differentiation:* given the expansion coefficients of a function, it should be easy to determine the set of coefficients associated with an approximation of a spatial derivative of the function.

For non-periodic boundary problems, algebraic polynomial functions are used, in the form of orthogonal (with respect to a weighted $L^2$-scalar product) systems of polynomials functions over the interval $(-1,1)$.

The present notebook aims at investigating some computational and numerical aspects of the [Lagrange interpolating polynomial](https://en.wikipedia.org/wiki/Lagrange_polynomial) of a function at the so-called Chebyshev-Gauss-Lobatto points and its representation in the basis of [Chebyshev polynomials of the first kind](https://en.wikipedia.org/wiki/Chebyshev_polynomials). 

The <tt>numpy</tt>, <tt>scipy</tt> and <tt>matplotlib</tt> packages will be needed.

In [ ]:
import numpy as np
import scipy as scp

# To draw matplotlib plots within this notebook.
%matplotlib inline
import matplotlib.pyplot as plt

## Exercise 1. The Lagrange interpolation at the Chebyshev-Gauss-Lobatto points using the Chebyshev basis.

Given a non-zero integer $N$, the Chebyshev interpolation of a given function $u$ over the interval $[-1,1]$ consists in the construction of the Lagrange interpolating polynomial of degree $N$ at the Chebyshev-Gauss-Lobatto point, that is the polynomial function $I_Nu$ of degree $N$ satisfying the conditions
$$
I_Nu(x_j)=u(x_j),\ j=0,\dots,N,
$$
at the Chebyshev-Gauss-Lobatto quadrature nodes
$$
x_j=\cos\left(\frac{\pi j}{N}\right),\ j=0,\dots,N.
$$

### The Chebyshev basis
When used in collocation spectral methods, this interpolation polynomial is written in the basis formed by the Chebyshev polynomials of the first kind, which is orthogonal with respect to the weighted $L^2_w((-1,1),\mathbb{R})$-scalar product, with weight $w(x)=\frac{1}{\sqrt{1-x^2}}$. They are the unique polynomials satisfying
$$
\forall k\in\mathbb{N},\ \forall\theta\in\mathbb{R},\ T_k(\cos(\theta))=\cos(k\theta).
$$
In practice, they can be obtained from the recurrence relation
$$
\begin{align*}
&T_0(x) = 1,\\
&T_1(x) = x,\\
&\forall k\in\mathbb{N}^*,\ T_{k+1}(x) = 2xT_{k}(x)-T_{k-1}(x).
\end{align*}
$$

Note that the Chebyshev-Gauss-Lobatto quadrature nodes introduced above are the extremum points of $T_N$ on the interval $[-1,1]$.

**Question.** Write a function computing the coefficients in the canonical basis of $\mathbb{P}_N$ of the $N+1$ first Chebyshev polynomials, the non-zero integer $N$ being given. The coefficients will be returned in a two-dimensional array.

In [ ]:
def chebyshev_polynomials(N):
    coefficients=np.zeros((N+1,N+1))
    coefficients[0,0],coefficients[1,1]=1,1
    for k in range(1,N):
        coefficients[k+1,1:]+=2*coefficients[k,:-1]
        coefficients[k+1]-=coefficients[k-1]
    return coefficients

**Question.** Using the previous function and the `polyval` function in the `polynomial.polynomial` library of <tt>numpy</tt>, plot the graphs of the first six Chebyshev polynomial functions over $[-1,1]$.

In [ ]:
from numpy.polynomial.polynomial import polyval

N=5
polynomials=chebyshev_polynomials(N)
x=np.linspace(-1,1,100)
for k in range(N+1):
    plt.plot(x,polyval(x,polynomials[k]),label=r'$T_{'+str(k)+'}(x)$')
plt.legend()

### The Chebyshev representation of the Lagrange interpolation polynomial at the Chebyshev nodes.

We now consider the Lagrange interpolation of a function $u$ defined on the interval $[-1,1]$ (the procedure can be generalised to a function defined on any compact domain $[a,b]$ through translation and scaling).

The interpolation is done at the Chebyshev-Gauss-Lobatto points previously introduced, and the interpolation polynomial is written in the Chebyshev basis:
$$
I_Nu(x)=\sum_{k=0}^N\tilde{u}_kT_k(x).
$$

**Question.** Provide an explicit form for the polynomial expansion coefficients $\tilde{u}_k$ and show that they can be computed using the type-I [discrete cosine transform](https://en.wikipedia.org/wiki/Discrete_cosine_transform), a Fourier-related transform similar to the discrete Fourier transform.

**Answer.** We first observe that, according to the respective definitions of the Chebyshev-Gauss-Lobatto points and of the Chebyshev polynomials, one has
$$
\forall k\in\mathbb{N},\ \forall j\in\{0,\dots,N\}, T_k(x_j)=\cos\left(k\frac{j\pi}{N}\right).
$$
Since the weights of the Chebyshev-Gauss-Lobatto formula are given by $w_j=\frac{\pi}{2N}$ for $j=0$ or $N$ and $w_j=\frac{\pi}{N}$ for $j=1,\dots,N-1$, it follows that 
$$
\forall k\in\{0,\dots,N\},\ \tilde{u}_k=\frac{1}{\gamma_k}\sum_{j=0}^Nu(x_j)T_k(x_j)w_j=\frac{\pi}{N\gamma_k}\left(\frac{1}{2}\left(u(x_0)+(-1)^ku(x_N)\right)+\sum_{j=1}^{N-1}u(x_j)\cos\left(k\frac{j\pi}{N}\right)\right).
$$
Finally, using that $\gamma_k=\pi$ if $k=0$ or $N$ and $\gamma_k=\frac{\pi}{2}$ if $k=1,\dots,N-1$, one reaches
$$
\tilde{u}_k=\begin{cases}\frac{1}{N}\left(\frac{1}{2}\left(u(x_0)+(-1)^ku(x_N)\right)+\sum_{j=1}^{N-1}u(x_j)\cos\left(k\frac{j\pi}{N}\right)\right)&\text{if }k=0\text{ or }N\\
\frac{2}{N}\left(\frac{1}{2}\left(u(x_0)+(-1)^ku(x_N)\right)+\sum_{j=1}^{N-1}u(x_j)\cos\left(k\frac{j\pi}{N}\right)\right)&\text{if }k=1,\dots,N-1\end{cases}.
$$
We thus see that the discrete polynomial coefficients $\tilde{u}_k$ are given by the type-I discrete cosine transform of the sequence $\{u(x_j)\}_{j=0,\dots,N}$, up to a multiplication by a scalar ($\frac{1}{N}$ for $k=0$ or $N$ and $\frac{2}{N}$ for $k=1,\dots,N-1$).

**Question.** Write a function computing the expansion coefficients of the interpolant $I_Nu$ of a function $u$, the function and the integer $N$ being given, using the `dct` function of the `fft` library of <tt>scipy</tt> (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.dct.html) and note the normalisation used in the definition of the DCT-I implemented).

In [ ]:
from scipy import fft

def chebyshev_interpolant_coefficients(u,N):
    x=np.cos(np.pi*np.arange(0,N+1)/N)
    coefficients=scp.fft.dct(u(x),type=1)/N
    coefficients[0]=coefficients[0]/2.
    coefficients[-1]=coefficients[-1]/2.
    return coefficients

**Question.** Write a function which evaluates the interpolant $I_Nu$ of a function $u$ at a given set of points, the set of coefficients of the interpolant being given.

In [ ]:
def evaluate_chebyshev_interpolant(coefficients,points):
    N=len(coefficients)-1
    polynomials=chebyshev_polynomials(N)
    values=np.zeros(len(points))
    for k in range(N+1):
        values+=coefficients[k]*polyval(points,polynomials[k])
    return values

**Question.** Use the written functions to plot and compare the graphs of the following functions and their respective interpolants over $[-1,1]$, for several values of $N$,

* $u(x) = \cos((x + 1)\pi) + \sin(2(x + 1)\pi)$,
* $u(x) = \mathbb{1}_{\left[-\frac{1}{2},\frac{1}{2}\right]}(x)$,
* $u(x) = \dfrac{1}{1+25x^2}$.

For which of these the Chebyshev interpolant seems to provide a relevant approximation of the function? Is the [Gibbs phenomenon](https://en.wikipedia.org/wiki/Gibbs_phenomenon) observed?

In [ ]:
N_values=[5,10,20,40]

y=np.linspace(-1,1,1000)

fig,axs=plt.subplots(4,3,figsize=(15,15))

for i,N in enumerate(N_values):
    axs[i,0].plot(y,np.cos((y+1.)*np.pi)+np.sin(2.*(y+1.)*np.pi),label=r'$\cos((x + 1)\pi)+\sin(2(x + 1)\pi)$')
    axs[i,0].plot(y,evaluate_chebyshev_interpolant(chebyshev_interpolant_coefficients(lambda x:np.cos((x+1.)*np.pi)+np.sin(2.*(x+1.)*np.pi),N),y),label='Chebyshev interpolant')
    axs[i,0].set_title('N='+str(N))
    axs[i,0].legend()

    axs[i,1].plot(y,(y>=-0.5) & (y<=0.5),label=r'$\mathbb{1}_{\left[-\frac{1}{2},\frac{1}{2}\right]}(x)$')
    axs[i,1].plot(y,evaluate_chebyshev_interpolant(chebyshev_interpolant_coefficients(lambda x:(x>=-0.5) & (x<=0.5),N),y),label='Chebyshev interpolant')
    axs[i,1].set_title('N='+str(N))
    axs[i,1].legend()

    axs[i,2].plot(y,1./(1.+25.*y**2),label=r'$\frac{1}{1+25x^2}$')
    axs[i,2].plot(y,evaluate_chebyshev_interpolant(chebyshev_interpolant_coefficients(lambda x:1./(1.+25.*x**2),N),y),label='Chebyshev interpolant')
    axs[i,2].set_title('N='+str(N))
    axs[i,2].legend()

fig.tight_layout()

**Answer.** The first and last function appear to be correctly approximated. However, the Gibbs phenomenon is observed for the second function, which presents discontinuities.

## Exercise 2. The Chebyshev interpolation derivative.

The *Chebyshev interpolation derivative* of a function $u$ is defined as the derivative of the interpolant $I_nu$, that is
$$
\mathcal{D}_Nu=(I_Nu)',
$$
and, using the representation in the Chebyshev basis previously used, one can write
$$
\mathcal{D}_Nu(x)= \sum_{k=0}^N\tilde{u}_k{T_k}'(x).
$$

**Question.** Show that
$$
\forall x\in(-1,1),\ (I_Nu)'(x) = \frac{1}{\sqrt{1-x^2}}\sum_{k=0}^Nk\tilde{u}_k\sin(k\arccos(x)),
$$
and, using l'Hôpital's rule, that
$$
(I_Nu)'(1)=\sum_{k=0}^Nk^2\tilde{u}_k,\\
(I_Nu)'(-1)=\sum_{k=0}^N(-1)^{k+1}k^2\tilde{u}_k.
$$

**Answer.** The first formula directly follows from the chain rule and the fact that the derivative of the arccosine function is $(\arccos)'(x)=-\frac{1}{\sqrt{1-x^2}}$. For the second and third formulas, it is first observed that
$$
\forall k\in\{0,\dots,N\},\ \lim_{x\to\pm1}\sin(k\arccos(x))=0\text{ and }\lim_{x\to\pm1}\sqrt{1-x^2}=0.
$$
Using l'Hôpital's rule, one gets
$$
\forall k\in\{0,\dots,N\},\ \lim_{x\to1}\frac{\sin(k\arccos(x))}{\sqrt{1-x^2}}=\lim_{x\to1}\frac{k\cos(k\arccos(x))}{x}=k
$$
and
$$
\forall k\in\{0,\dots,N\},\ \lim_{x\to-1}\frac{\sin(k\arccos(x))}{\sqrt{1-x^2}}=\lim_{x\to-1}\frac{k\cos(k\arccos(x))}{x}=(-1)^{k+1}k.
$$

We recall that the type-I [discrete sine transform](https://en.wikipedia.org/wiki/Discrete_sine_transform) of the sequence $\{v_i\}_{i=0,\dots,M}$ is the sequence $\{\tilde{v}_m\}_{m=0,\dots,M}$ defined by
$$
\forall m\in\{0,\dots,M\},\ \tilde{v}_m=\sum_{i=0}^{M}v_i\sin\left(\frac{\pi}{M+2}(i+1)(m+1)\right).
$$

**Question.** Write a function which computes the values $(I_Nu)'(x_j)$, $j=0,\dots,N$, the coefficients of $I_Nu$ being given, using the `idst` function of the `fft` library of <tt>scipy</tt> (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.idst.html)) returning the inverse [discrete sine transform](https://en.wikipedia.org/wiki/Discrete_sine_transform) of a sequence.

**Answer.** In view of the above formula and of the definition of the Chebyshev nodes, one has
$$
\forall j\in\{1,\dots,N-1\},\ (I_Nu)'(x_j)=\frac{1}{\sqrt{1-{x_j}^2}}\sum_{k=1}^{N-1}k\tilde{u}_k\sin\left(k\frac{j\pi}{N}\right).
$$
Using a change of indices and setting $M=N-2$, one then gets
$$
\forall i\in\{0,\dots,M\},\ (I_Nu)'(x_{i+1})=\frac{1}{\sqrt{1-{x_{i+1}}^2}}\sum_{m=0}^M(m+1)\tilde{u}_{m+1}\sin\left((m+1)\frac{(i+1)\pi}{M+2}\right),
$$
and the values of $(I_Nu)'(x_j)$ for $j=1,\dots,N-1$ can be computed using an inverse discrete sine transform.
Finally, it follows from the last two formulas in the preceding question that
$$
(I_Nu)'(x_0)=\sum_{k=0}^Nk^2\tilde{u}_k\text{ and }(I_Nu)'(x_N)=\sum_{k=0}^N(-1)^{k+1}k^2\tilde{u}_k.
$$

In [ ]:
def chebyshev_differentiation_values(coefficients):
    N=len(coefficients)-1
    values=np.zeros(N+1)
    k=np.arange(N+1)
    values[0]=np.dot(k**2,coefficients)
    values[1:-1]=N*scp.fft.idst(k[1:-1]*coefficients[1:-1],type=1)/np.sqrt(1.-(np.cos(np.pi*k/N)[1:-1])**2)
    values[-1]=np.dot(k**2*np.power(-np.ones(N+1),k+1),coefficients)
    return values

**Question.** Compare the graphs of the derivatives the following functions with the values at the interpolation nodes of their respective Chebyshev interpolation derivatives, for several values of $N$:

* $u(x) = \cos((x + 1)\pi) + \sin(2(x + 1)\pi)$,
* $u(x) = \begin{cases}1 & \text{if } -1 \leq x < 0 \\-1 & \text{if } 0 \leq x \leq 1\end{cases}$,
* $u(x) = \dfrac{1}{1+25x^2}$.

In [ ]:
N_values=[5,10,20,40]

y=np.linspace(-1,1,1000)

fig,axs=plt.subplots(4,3,figsize=(15,15))

for i,N in enumerate(N_values):
    
    x=np.cos(np.pi*np.arange(0,N+1)/N)
    
    axs[i,0].plot(y,np.pi*(-np.sin((y+1.)*np.pi)+2.*np.cos(2.*(y+1.)*np.pi)),label=r'$\pi(2\sin(2(x + 1)\pi)-\sin((x + 1)\pi))$')
    axs[i,0].plot(x,chebyshev_differentiation_values(chebyshev_interpolant_coefficients(lambda x:np.cos((x+1.)*np.pi)+np.sin(2.*(x+1.)*np.pi),N)),'*',label='Chebyshev interpolation derivative')
    axs[i,0].set_title('N='+str(N))
    axs[i,0].legend()

    axs[i,1].plot(y,np.zeros(1000),label=r'$0$')
    axs[i,1].plot(x,chebyshev_differentiation_values(chebyshev_interpolant_coefficients(lambda x:(x>=-0.5) & (x<=0.5),N)),'*',label='Chebyshev interpolation derivative')
    axs[i,1].set_title('N='+str(N))
    axs[i,1].legend()

    axs[i,2].plot(y,-50*y/(1+25*y**2)**2,label=r'$\frac{-50 y}{(1+25 y^2)^2}$')
    axs[i,2].plot(x,chebyshev_differentiation_values(chebyshev_interpolant_coefficients(lambda x:1/(1+25*x**2),N)),'*',label='Chebyshev interpolation derivative')
    axs[i,2].set_title('N='+str(N))
    axs[i,2].legend()

fig.tight_layout()

## Exercise 3. Interpolation at equidistant nodes and the Runge phenomenon.

In this exercise, the use of the [Chebyshev nodes](https://en.wikipedia.org/wiki/Chebyshev_nodes) for the Lagrange interpolation of a function is motivated by observing a problem occuring with evenly spaced nodes: the so called [Runge phenomenon](https://en.wikipedia.org/wiki/Runge%27s_phenomenon).

Consider the approximation of the function $u(x)=\frac{1}{1 + 25x^2}$ over the interval $[-1,1]$ by its Lagrange interpolation polynomial associated with the equidistant nodes
$$
x_j=-1+\frac{2j}{N},\ j=0,\dots,N,
$$
where $N$ is a non-zero natural integer.

If $N$ is not large, the representation of such a polynomial in the canonical basis of $\mathbb{P}_N$ can be computed using the `lagrange` function in the `interpolate` library of <tt>scipy</tt> (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.lagrange.html)).

**Question.** Compare the graphs over the interval $[-1,1]$ of the function $u$ and of its interpolation polynomial $I_Nu$ at equidistributed nodes for several values of $N$.

In [ ]:
from scipy import interpolate

def u(x):
    return 1./(1.+25.*x**2)

y=np.linspace(-1,1,1000)

plt.plot(y,u(y),label=r'$u(x)$')

N_values=[5,10,15,20]

for N in N_values:
    x=np.linspace(-1,1,N)
    plt.plot(y,polyval(y,scp.interpolate.lagrange(x,u(x)).coef[::-1]),label=r'$I_{'+str(N)+'}u(x)$')
plt.legend()

**Question.** What happens when the interpolant degree $N$ is increased? Conjecture on the convergence of the sequence of interpolation polynomials of the function and conclude on the adequacy of the choice of evenly spaced nodes for Lagrange interpolation.

**Answer.** The interpolation polynomials do not converge to the function as $N$ is increased, as the amplitude of the oscillations observed near the endpoints of the interval augment considerably with $N$. One concludes that high-degree polynomial interpolation at equidistant points can be troublesome.

**Question.** Compare the graphs over the interval $[-1,1]$ of the function $u$ and of its interpolation polynomial $I_Nu$ at the [Chebyshev nodes](https://en.wikipedia.org/wiki/Chebyshev_nodes) 
$$
x_j=\cos\left(\frac{2j+1}{2(N+1)}\,\pi\right),\ j=0,\dots,N,
$$
which are is the roots of $T_{n+1}$, for several values of $N$. Conclude.

In [ ]:
plt.plot(y,u(y),label=r'$u(x)$')

for N in N_values:
    x=np.cos(0.5*np.pi*(2*np.arange(N+1)+np.ones(N+1))/(N+1))
    plt.plot(y,polyval(y,scp.interpolate.lagrange(x,u(x)).coef[::-1]),label=r'$I_{'+str(N)+'}u(x)$')
plt.legend()

**Answer.** With this choice of interpolation nodes, the sequence appears to uniformly converge to the function.